# HW7

In [1]:
from IPython.display import Image 
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy import io
import scipy.ndimage
import scipy.misc

np.set_printoptions(threshold=np.nan)

import sklearn
%matplotlib inline

# NP FNS

In [2]:
def vec(arr): # d b 1 --> (d, 1)
    return arr.reshape((arr.shape[0], 1))

In [3]:
def arr(vec): # 1 by d --> (d, )
    return vec.flatten()

# Problem 1: Lloyd's Method on MNIST

In [4]:
dict_mnist_data = sp.io.loadmat("mnist_data/images.mat")
points = dict_mnist_data['images']

In [5]:
'''

    THIS WAS STOLEN (I mean borrowed) FROM http://flothesof.github.io/k-means-numpy.html

'''
def kMeans(points, k, numIter):
    centroids = initialize_centroids(points, k)
    for i in range(numIter):
        lstIndices2ClosestCentroid = closest_centroid(points, centroids)
        centroids = move_centroids(points, lstIndices2ClosestCentroid, centroids)
    return centroids

def initialize_centroids(points, k):
    """returns k centroids from the initial points"""
    centroids = points.copy()
    np.random.shuffle(centroids)
    return centroids[:k]


def closest_centroid(points, centroids):
    """returns an array containing the index to the nearest centroid for each point"""
    distances = np.sqrt(((points - centroids[:, np.newaxis])**2).sum(axis=2))
    return np.argmin(distances, axis=0)

def move_centroids(points, closest, centroids):
    """returns the new centroids assigned from the points closest to them"""
    return np.array([points[closest==k].mean(axis=0) for k in range(centroids.shape[0])])

def visualize_centroids(centroids):
    i = 0
    for c in centroids:
        plt.title('Centroid {label}'.format(label=i))
        c = c.reshape((28, 28))
        plt.imshow(c, cmap='gray')
        plt.show()
        i+=1

In [6]:
# points = np.reshape(points, (784, 60000)).T
# centroids = kMeans(points, 5, 5)
# visualize_centroids(centroids)

# Problem 2: Low Rank Approximation

In [8]:
face = sp.ndimage.imread("hw7_data/low_rank_data/face.jpg")
sky = scipy.ndimage.imread("hw7_data/low-rank_data/sky.jpg")

ImportError: Could not import the Python Imaging Library (PIL) required to load image files.  Please refer to http://pillow.readthedocs.org/en/latest/installation.html for installation instructions.

In [ ]:
def reduceArray(image, k):
    U, S, V = np.linalg.svd(image, full_matrices=False)
    U = U[:, :k]
    print("U " + str(U.shape))
    S = S[:k]
    print("S " + str(S.shape))
    V = V[:k]
    print("V " + str(V.shape))
    return U.dot(diag(S).dot(V))

def computeMSE(original, reduced):
    return np.sum((original-reduced)**2)/original.size

def approxImage(image, lstRanks, name):
    '''
        Displays img approximation of rank k and plots MSE's 
    '''
    errors = []
    for i in lstRanks:
        reduction = reduceArray[image, i]
        plt.title('Rank {i} Reduction'.format(label=i))
        plt.imshow(reduction)
        plt.show()
    #     errors.append(computeMSE(face, reduction))
        errors.append(np.linalg.np(face-reduction))
    plt.plot(errors)
    plt.title('MSE of IMG Reduction on ' + name)
    plt.show() 

In [ ]:
ranks = [5, 20, 100]
approxImage(face, ranks, 'Face')
approxImage(sky, ranks, 'Sky')

# Problem 3